# exemple de régression logistique avec $sklearn$

## on importe les bibliothèques

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

## on lit les données

In [ ]:
filename = "exam.csv"

In [ ]:
columns = ['first_exam', 'second_exam', 'admitted']

In [ ]:
exam = pd.read_csv(filename, names=columns, header=None)

In [ ]:
exam.head()

In [ ]:
exam[['first_exam', 'second_exam']].describe()

## on plot les histogrammes

In [ ]:
exam[['first_exam', 'second_exam', 'admitted']].hist(figsize=(5, 5), bins=15);

## on plot les boîtes à moustache

In [ ]:
exam[['first_exam', 'second_exam']].boxplot(figsize=(4, 2))

## on compte les élèves au dessous et au dessus de la moyenne

### premier examen

In [ ]:
np.sum(exam['first_exam'] < 50), np.sum(exam['first_exam'] >= 50)

### second examen 

In [ ]:
np.count_nonzero(exam['second_exam'] < 50), np.count_nonzero(exam['second_exam'] >= 50)

## on plot les notes

### avec le paramètre `c` de `scatter`

In [ ]:
scatter = plt.scatter(exam['first_exam'], exam['second_exam'], c=exam['admitted'], cmap="winter",
                     marker='.');
plt.xlabel('first_exam')
plt.ylabel('second_exam')

plt.legend(*scatter.legend_elements()); # les labels pour la légende;

## on crée un modèle de régression logistique

Le modèle est un modèle linéaire avec deux variables (les notes des deux examens) et un terme constant
$$\theta_0 + \theta_1\;  note\_first\_exam + \theta_2\;  note\_second\_exam = 0$$  
on va rechercher par l'apprentissage les coefficients $\theta_0$, $\theta_1$ et $\theta_2$ de la droite qui sépare le mieux les deux nuages de points `admitted` et `refused`

In [ ]:
plt.scatter(exam['first_exam'], exam['second_exam'], c=exam['admitted'], marker='.')
plt.plot([40, 100], [100, 36]);
plt.plot([30, 80], [100, 30]);

on recherche la droite qui fait la plus petite erreur  
qui classifie bien un maximum de points

In [ ]:
from sklearn import linear_model

on crée un objet de type régression logistique

In [ ]:
model = linear_model.LogisticRegression(solver='newton-cg') # par exemple newton avec gradien conjugué

notons que vous allez pouvoir fixer un grand nombre de paramètres dont le solver, la fonction de perte... (voir le help de `linear_model.LogisticRegression`

In [ ]:
#linear_model.LogisticRegression?

## les données d'entrée et de sorties

In [ ]:
X = exam[['first_exam', 'second_exam']]
y = exam['admitted']

## on `fit` le modèle

pour déterminer les $\theta_i$

In [ ]:
model.fit(X, y);

### le terme constant

In [ ]:
model.intercept_ # theta0

### les coefficients

In [ ]:
model.coef_ # theta1 et theta2

### équation de la droite qui sépare les deux classes $admitted$ et $refused$

   - $\theta_0 + \theta_1\;  note\_first\_exam + \theta_2\;  note\_second\_exam = 0$

où

   - $\theta_0 = model.intercept\_[0]$
   - $\theta_1 = model.coef\_[0][0]$
   - $\theta_2 = model.coef\_[0][1]$

donc $y = -(\theta_0 + \theta_1 x))/\theta_2$

In [ ]:
def y_line(x):
    return -(model.intercept_[0] + model.coef_[0][0]*x)/model.coef_[0][1]

la fonction `y_line` s'applique à un argument, on la vectorise pour qu'elle s'applique à un vecteur d'arguments

In [ ]:
y_line_vect = np.vectorize(y_line)

### on plot la fonction de prédiction trouvée

In [ ]:
plt.scatter(exam['first_exam'], exam['second_exam'], c=exam['admitted'], marker='.')
plt.plot([30, 100], y_line_vect([30, 100]));

## les prédictions

### calcul *à-la-main*

Dans nos prédictions, les admis sont les points au dessus de la droite et les refusés sont les points au dessous de la droite

les admis sont les points tels que  
$\theta_0 + \theta_1\;  note\_first\_exam + \theta_2\;  note\_second\_exam >= 0$  

les refusés sont les points tels que  
$\theta_0 + \theta_1\;  note\_first\_exam + \theta_2\;  note\_second\_exam < 0$

In [ ]:
exam['admitted'][0:2] # les 2 premiers

In [ ]:
predicted = (model.intercept_[0] 
             + model.coef_[0][0] * exam['first_exam']
             + model.coef_[0][1] * exam['second_exam']) >= 0

In [ ]:
predicted

### calcul avec $sklearn$ 

naturellement on peut laisser `sklearn` calculer les prédictions

In [ ]:
y_predict = model.predict(X)

on vérifie que ce sont les mêmes

In [ ]:
np.all(y_predict == predicted)

### on trace les prédictions

In [ ]:
# les mesurés sont en vert et rouge

admitted = exam.loc[exam['admitted'] == 1]
refused  = exam.loc[exam['admitted'] == 0]

plt.plot(admitted['first_exam'], admitted['second_exam'], 'go', label='admitted')
plt.plot(refused['first_exam'],  refused['second_exam'],  'ro', label='refused')

# les prédits sont en bleu et jaune

admitted_predict = exam.loc[y_predict == 1]
refused_predict = exam.loc[y_predict == 0]

plt.plot(admitted_predict['first_exam'], admitted_predict['second_exam'], 'b.', label='predict admitted')
plt.plot(refused_predict['first_exam'],  refused_predict['second_exam'],  'y.', label='predict refused')

on voit de bonnes prédictions
* bleu cerclés de vert sont les bien prédits `admitted`
* jaunes cerclés de rouge sont les bien prédits `refused`

on voit des erreurs à la frontière
* les jaunes cerclés de vert sont les prédits refusés mais en fait admis
* les bleu cerclés de rouge sont les prédits acceptés mais en fait refusés

... on en discute juste après

## les erreurs

### on peut calculer le taux d'erreur à-la-main

le nombre des mauvaises classifications toutes classes confondues

In [ ]:
np.sum(exam['admitted'] == y_predict)/100

### on peut le calculer avec `sklearn`

In [ ]:
model.score(X, y)

## les résultats

nos prédictions

In [ ]:
y_predict = model.predict(X)

### les vrais positifs

on les a prédits `admitted` (`1`) ils étaient `admitted` (`1`)

In [ ]:
true_positive = exam.loc[(y_predict == 1) & (exam['admitted'] == 1)]

In [ ]:
len(true_positive)

### les vrais négatifs

on les a prédits `refused` (`0`) ils étaient `refused` (`0`)

In [ ]:
true_negative = exam.loc[(y_predict == 0) & (exam['admitted'] == 0)]

In [ ]:
len(true_negative)

### les faux positifs

on les a prédits `admitted` (`1`) alors qu'ils étaient `refused` (`0`)

In [ ]:
false_positive = exam.loc[(y_predict == 1) & (exam['admitted'] == 0)]

In [ ]:
len(false_positive)

### les faux négatifs
on les a prédits `refused` (`0`) alors qu'ils étaient `admitted` (`1`)

In [ ]:
false_negative = exam.loc[(y_predict == 0) & (exam['admitted'] == 1)]

In [ ]:
len(false_negative)

### matrice de confusion

la matrice de confusion de notre exemple:

| | admitted | refused|
| --| --| --|
| predict admitted |  55 | 6 | 
| predict refused | 5 | 34 |

donne une estimation de la qualité de notre modèle de classification

**plusieurs mesures statistiques sont définies**:

**True positive rate**, **Recall**, **Sensitivity**, **probability of detection** $= \dfrac{vrai \, positif}{vrai\, positif\, +\, faux\, négatif}$ 




**False positive rate**, **Fall-out**, **probability of false alarm** $= \dfrac{faux\, positif}
{faux\, positif\, +\, vrai\, négatif}$

### avec `sklearn`

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
C = confusion_matrix(y_true=exam['admitted'], y_pred=y_predict)

In [ ]:
C

In [ ]:
C[0, 0] # vrais négatifs

In [ ]:
C[0, 1] # faux positifs

In [ ]:
C[1, 0] # faux négatifs

In [ ]:
C[1, 1] # vrais positifs

END